In [222]:
import os
import re
import openai
import json
import psycopg2
from dotenv import load_dotenv
from sqlalchemy import create_engine, inspect
from sqlalchemy.engine import URL
from langchain.chat_models import ChatOpenAI
import tiktoken
from langchain_community.utilities import SQLDatabase
from langchain.chains import create_sql_query_chain
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter
from langchain_openai import ChatOpenAI
load_dotenv(override = True)

True

In [340]:
user = os.getenv('USER')
host = os.getenv('HOST')
port = os.getenv('PORT')
password = os.getenv('PASSWORD')
dbname = os.getenv('DB1')

# Set up PostgreSQL connection
connection = psycopg2.connect(
    host=host,
    port=port,
    dbname=dbname,
    user=user,
    password=password
)
cursor = connection.cursor()

In [225]:
sql_query_acc_name = """
SELECT accname, accno, report
FROM "ACCNO";
"""

try:
    # Execute the query
    cursor.execute(sql_query_acc_name)
    
    # Fetch the results
    acc_name = cursor.fetchall()
except psycopg2.Error as e:
    print(f"Database error: {e}")
    connection.rollback()

In [341]:
sqlalchemy_connection_url = URL.create(
    "postgresql+psycopg2",
    username=user,
    password=password,
    host=host,
    database=dbname,
)
sql_engine = create_engine(sqlalchemy_connection_url)

In [342]:
openai_api_key = os.getenv("OPENAI_API_KEY")
openai.api_key = openai_api_key

In [239]:
with open('db_structure.json', 'r', encoding='utf-8') as file:
    db_structure_data = json.load(file)

In [343]:
import time

In [344]:
class GetAnswer:
    def __init__(self, model="gpt-4o-mini", engine=None, db_structure=None, acc_name=None):
        self.model = model
        self.llm = ChatOpenAI(model=model)
        self.sql_engine = engine
        self.sql_database = SQLDatabase(engine=engine)
        self.db_structure = db_structure
        self.acc_name = acc_name

#Lower name để tìm đc, embedding name
    def chain_of_thought_prompt(self, question):
        prompt = f"""
    Bạn là một chuyên gia phân tích tài chính chuyên sâu. Hãy phân tích câu hỏi dưới đây và thực hiện các bước để trả lời chính xác và chi tiết.

    Câu hỏi: "{question}"

    Bước 1: Xác định loại vấn đề cần phân tích và chia nhỏ vấn đề
    - Dịch câu hỏi sang tiếng anh
    - Phân tích yêu cầu và chia nhỏ bài toán: [Xác định các vấn đề cụ thể cần được phân tích từ câu hỏi, chẳng hạn như các chỉ số tài chính hoặc thông tin tài sản liên quan]
    - Từ đó, hãy tạo 1 danh sách với các câu lệnh nhỏ hơn với các vấn đề nhỏ đã được chia
    Ví dụ: ['Hãy tìm lợi nhuận sau thuế và tổng vốn rồi tính tỷ lệ ROE rồi tìm ngân hàng có ROE lớn nhất năm 2024 quý 2","Hãy tìm tổng tài sản của ACB năm 2024 quý 2"]

    Bước 2: Xác định thông tin cơ bản cần truy vấn từ vấn đề ở Bước 1
    - Tên công ty: [Liệt kê rõ những công ty được đề cập, ví dụ: BIDV, Vietcombank, ACB]
    Lưu ý: Luôn quy chuẩn từ Ngân hàng về NH. Ví dụ: 'Ngân hàng Việt Nam Thịnh Vượng' thành 'NH Việt Nam Thịnh Vượng'
    - Năm tài chính: [Ghi rõ năm tài chính cần phân tích, ví dụ: 2024]
    - Kỳ báo cáo: [Xác định kỳ báo cáo cụ thể, chẳng hạn quý 1, quý 2, quý 3, quý 4]
    - Mục cụ thể trong báo cáo: [Dựa vào các vấn đề đã được chia nhỏ, xác định rõ mục cần tìm trong báo cáo tài chính. Ví dụ: doanh thu, lợi nhuận ròng, tổng tài sản, ROE (Return on Equity). Nếu liên quan đến các chỉ số tài chính, xác định các thành phần cụ thể để tính toán, ví dụ: lợi nhuận sau thuế, vốn chủ sở hữu]
    - Loại báo cáo: [Xác định rõ loại báo cáo tài chính cần thiết, chẳng hạn Bảng cân đối kế toán, Báo cáo kết quả kinh doanh, Báo cáo lưu chuyển tiền tệ]

    Bước 3: Trả về câu truy vấn SQL dựa trên các thông tin đã xác định ở bước 1, 2
    - Với từng vấn đề trong danh sách trên, hãy tạo từng câu truy vấn sql cho từng vấn đề nhỏ
    - Cấu trúc database của bạn: {self.db_structure}
    - Danh sách tên các tài khoản có sẵn: {json.dumps(self.acc_name)}
    Đây là ví dụ: {self.load_example()}

    Bước 4: Giải thích và kiểm tra lại lỗi truy vấn SQL
    - Với từng vấn đề trong danh sách trên, hãy giải thích lại lệnh truy vấn.
    - Sau khi giải thích, hãy đối chiếu lại với yêu cầu của câu hỏi và xác nhận câu truy vấn đấy thỏa mãn câu hỏi chưa
    - Nếu đúng thì xác nhận và chuyển sang bước 5, nếu sai thì hãy sửa lại câu truy vấn đấy bằng cách thực hiện lại từ bước 1

    Bước 5: Ghi lại câu truy vấn SQL cuối cùng 
    - Sau khi đảm bảo các câu truy vấn đã đúng, Luôn viết lại câu truy vấn theo fomart sau:
   ```sql[câu truy vấn SQL chính xác]```
    Lưu ý: 
    - Chỉ sử dụng dữ liệu từ database đã kết nối và đảm bảo truy vấn chính xác từng chỉ số tài chính được yêu cầu.
    - Hầu hết các tên ngân hàng viết tắt đều ở symbol, nếu không được hãy thử sang abbreviation nếu câu hỏi vẫn yêu cầu tên viết tắt. 
    - bankname chứa tên tiếng việt, banknameng chứa tên tiếng anh
    - Với những câu hỏi tìm số tài khoản chung hoặc tìm tài khoản cấp X mà không đề cập đến thời gian/ngân hàng, hãy tìm trong bảng ACCNO
    - Bậc của số tài khoản được quyết định theo độ dài. 1 là cao nhất, càng dài càng thấp.(Ví dụ: tài khoản 10, 11, 12 là tài khoản con của 1)
    - Những câu hỏi liệt kê, Không tự động giới hạn kết quả, hãy liệt kê hết.
    - Luôn đảm bảo những câu query SQL phải trả kết quả về.
    Hãy làm tuần tự từ bước 1 và trả lời bằng tiếng việt
    """
        return prompt
    
    def get_token_usage(self, text):
        """
        Returns the estimated token usage for the given text.
        """
        encoding = tiktoken.encoding_for_model(self.model)  # Ensure the model is specified

        # Calculate the number of tokens for the text
        return len(encoding.encode(text))
    
    def load_example(self):
        """Loads the example text from example.txt."""
        with open('example.txt', 'r', encoding='utf-8') as file:
            return file.read()
        
    def get_sql_query(self, input_question):
        try:
            # Generate the prompt using the chain of thought method
            prompt_cot = self.chain_of_thought_prompt(input_question)
            # Create a chain to generate the SQL query
            sql_query_chain = create_sql_query_chain(self.llm, self.sql_database)
            # Invoke the chain and get the SQL query
            sql_query = sql_query_chain.invoke({"question": prompt_cot})
            # Extract the SQL query from the response
            return self.extract_sql_queries(sql_query)
        except Exception as e:
            print(f"Error generating SQL query: {e}")
            return None

    def extract_sql_queries(self, sql_query):
        """
        Extracts all SQL queries from the response and returns them in a list.
        """
        pattern = r"```sql\s*(.*?)\s*```"
        matches = re.findall(pattern, sql_query, re.DOTALL)
        # Ensure unique queries by converting to a set and back to a list
        return list(set(match.strip() for match in matches)) if matches else []

    def execute_sql_queries(self, sql_queries):
        """
        Execute each SQL query in the list and return the results in the order they were executed.
        """
        results = []  # List to store the results
        for query in sql_queries:
            try:
                # Execute the query using your database connection
                result = QuerySQLDataBaseTool(db=self.sql_database).run(query)
                results.append(result)  # Add the result to the list
            except Exception as e:
                print(f"Error executing query: {query}\nError: {e}")
                results.append(None)  # Add None if there is an error
                
        return results
    

    def get_result(self, question):
        system_role_template = """
        Given the following user question, corresponding SQL queries, and SQL results, provide a detailed answer.
        Question: {question}
        SQL Queries: {queries}
        SQL Results: {results}
        Answer:
        """

        total_input_tokens = 0
        total_output_tokens = 0

        try:
            start_time = time.time()

            # Step 1: Calculate tokens for the question
            prompt_cot = self.chain_of_thought_prompt(question)
            input_tokens_question = self.get_token_usage(prompt_cot)
            total_input_tokens += input_tokens_question
            
            print(f"Input tokens for question: {input_tokens_question}")

            # Step 2: Generate SQL queries from the question
            sql_queries = self.get_sql_query(question)
            input_tokens_sql = sum(self.get_token_usage(query) for query in sql_queries)
            total_input_tokens += input_tokens_sql
            print(f"Input tokens for SQL queries: {input_tokens_sql}")

            if not sql_queries:
                raise ValueError("Failed to generate SQL query from the question.")

            # Step 3: Execute all SQL queries and get results
            sql_results = self.execute_sql_queries(sql_queries)

            # Step 4: Format prompt and create the final answer
            answer_input = {
                "question": question,
                "queries": sql_queries,
                "results": sql_results
            }
            answer_prompt = PromptTemplate.from_template(system_role_template)
            answer_chain = answer_prompt | self.llm | StrOutputParser()

            # Step 5: Generate and return the answer
            answer = answer_chain.invoke(answer_input)
            output_tokens_answer = self.get_token_usage(answer)
            total_output_tokens += output_tokens_answer
            print(f"Output tokens for answer: {output_tokens_answer}")

            # Total tokens
            print(f"Total token:{total_input_tokens+total_output_tokens}")
            end_time = time.time()  # End timing
            execution_time = end_time - start_time
            print(f"Execution time for question: {execution_time:.2f} seconds")


            return answer

        except Exception as e:
            print(f"Error: {e}")
            return None

In [353]:
message = 'Dòng tiền vào từ hoạt động kinh doanh quý nào là cao nhất của MBB trong năm 2024, và lý do là gì?'
text2sql_instance = GetAnswer(model="gpt-4o-mini", engine=sql_engine, db_structure=db_structure_data, acc_name=acc_name)
try:
    final_answer = text2sql_instance.get_result(message)
    if final_answer:
        print(final_answer)
except KeyError as e:
    print("Error:", e)


e:\llm_fin_2\venv\Lib\site-packages\langchain_community\utilities\sql_database.py:123: SAWarning: Did not recognize type 'vector' of column 'embedding'
  self._metadata.reflect(
e:\llm_fin_2\venv\Lib\site-packages\langchain_community\utilities\sql_database.py:123: SAWarning: Did not recognize type 'vector' of column 'bankembedding'
  self._metadata.reflect(


Input tokens for question: 5123
Input tokens for SQL queries: 131
Output tokens for answer: 346
Total token:5600
Execution time for question: 23.99 seconds
Dòng tiền vào từ hoạt động kinh doanh quý cao nhất của MBB trong năm 2024 là quý 1, với tổng số tiền là 263,719.00. 

**Lý do:**
Mặc dù không có thông tin cụ thể về lý do dòng tiền vào từ hoạt động kinh doanh trong quý 1 cao hơn các quý khác, một số yếu tố có thể góp phần vào điều này, bao gồm:

1. **Mùa vụ kinh doanh:** Quý 1 thường là thời điểm mà các doanh nghiệp thực hiện nhiều hoạt động kinh doanh, đặc biệt là sau kỳ nghỉ Tết Nguyên Đán ở Việt Nam, khi mà các giao dịch thương mại bắt đầu tăng cao.

2. **Chính sách khuyến mãi:** MBB có thể triển khai các chương trình khuyến mãi hoặc sản phẩm mới trong quý 1, thu hút nhiều khách hàng và tạo ra dòng tiền vào lớn hơn.

3. **Tăng trưởng doanh thu:** Nếu MBB có sự tăng trưởng doanh thu mạnh mẽ trong quý 1, điều này tự nhiên sẽ dẫn đến dòng tiền vào từ hoạt động kinh doanh cao hơn.

4

In [361]:
message = 'Thu nhập từ đầu tư chứng khoán trong kỳ 3 của MBB năm 2024 có cao hơn kỳ trước không, và điều đó có liên quan đến các yếu tố thị trường nào?'
text2sql_instance = GetAnswer(model="gpt-4o-mini", engine=sql_engine, db_structure=db_structure_data, acc_name=acc_name)
try:
    final_answer = text2sql_instance.get_result(message)
    if final_answer:
        print(final_answer)
except KeyError as e:
    print("Error:", e)


e:\llm_fin_2\venv\Lib\site-packages\langchain_community\utilities\sql_database.py:123: SAWarning: Did not recognize type 'vector' of column 'embedding'
  self._metadata.reflect(
e:\llm_fin_2\venv\Lib\site-packages\langchain_community\utilities\sql_database.py:123: SAWarning: Did not recognize type 'vector' of column 'bankembedding'
  self._metadata.reflect(


Input tokens for question: 5134
Input tokens for SQL queries: 486
Output tokens for answer: 636
Total token:6256
Execution time for question: 30.03 seconds
To address the user's question regarding whether the income from securities trading in Q3 2024 for MBB is higher than in the previous quarter (Q2 2024), we first need to analyze the SQL queries executed and the results obtained.

### SQL Queries Breakdown:
1. **Query for Q3 2024 Income**:
   ```sql
   SELECT "INCOMESTATEMENT"."amount"
   FROM "METADATA"
   JOIN "INCOMESTATEMENT" ON "METADATA"."reportid" = "INCOMESTATEMENT"."reportid"
   WHERE "METADATA"."year" = 2024 
   AND "METADATA"."quarter" = 3 
   AND "INCOMESTATEMENT"."accountname" = 'Net profit from trading securities';
   ```
   - This query is designed to fetch the income from trading securities for MBB in Q3 2024.

2. **Query for Q2 2024 Income**:
   ```sql
   SELECT "INCOMESTATEMENT"."amount"
   FROM "METADATA"
   JOIN "INCOMESTATEMENT" ON "METADATA"."reportid" = "INCOME

In [365]:
message = 'Ngân hàng ACB quý 2 năm 2024 có xu hướng phụ thuộc vào một loại hình cho vay nào không (ví dụ: cho vay tiêu dùng, cho vay doanh nghiệp)?'
text2sql_instance = GetAnswer(model="gpt-4o-mini", engine=sql_engine, db_structure=db_structure_data, acc_name=acc_name)
try:
    final_answer = text2sql_instance.get_result(message)
    if final_answer:
        print(final_answer)
except KeyError as e:
    print("Error:", e)


e:\llm_fin_2\venv\Lib\site-packages\langchain_community\utilities\sql_database.py:123: SAWarning: Did not recognize type 'vector' of column 'embedding'
  self._metadata.reflect(
e:\llm_fin_2\venv\Lib\site-packages\langchain_community\utilities\sql_database.py:123: SAWarning: Did not recognize type 'vector' of column 'bankembedding'
  self._metadata.reflect(


Input tokens for question: 5134
Input tokens for SQL queries: 145
Output tokens for answer: 593
Total token:5872
Execution time for question: 48.82 seconds
To determine whether ACB Bank has a trend towards reliance on a specific type of loan in the second quarter of 2024, we need to analyze the SQL queries provided and their corresponding results.

### Analyzing SQL Queries

1. **Query 1**: 
   ```sql
   SELECT SUM("BALANCESHEET"."amount") AS "Total Loans"
   FROM "METADATA"
   JOIN "BALANCESHEET" ON "METADATA"."reportid" = "BALANCESHEET"."reportid"
   WHERE "METADATA"."year" = 2024 
   AND "METADATA"."quarter" = 2 
   AND "BALANCESHEET"."accountname" = 'Loans to customers';
   ```
   - **Purpose**: This query aims to calculate the total amount of loans given to customers by ACB Bank in Q2 2024.
   - **Result**: `[(Decimal('6411153444.00'),)]` indicates that the total loans to customers amount to 6,411,153,444.00.

2. **Query 2 and Query 3**:
   ```sql
   SELECT "ACCNO"."accname"
   FR

In [360]:
message = 'Tỷ lệ doanh thu từ các khoản vay thế chấp có tăng so với các khoản vay tiêu dùng không, và lý do của sự chuyển dịch này là gì? Hãy xét trong ngân hàng ACB quý 2 năm 2024'
text2sql_instance = GetAnswer(model="gpt-4o-mini", engine=sql_engine, db_structure=db_structure_data, acc_name=acc_name)
try:
    final_answer = text2sql_instance.get_result(message)
    if final_answer:
        print(final_answer)
except KeyError as e:
    print("Error:", e)


e:\llm_fin_2\venv\Lib\site-packages\langchain_community\utilities\sql_database.py:123: SAWarning: Did not recognize type 'vector' of column 'embedding'
  self._metadata.reflect(
e:\llm_fin_2\venv\Lib\site-packages\langchain_community\utilities\sql_database.py:123: SAWarning: Did not recognize type 'vector' of column 'bankembedding'
  self._metadata.reflect(


Input tokens for question: 5144
Input tokens for SQL queries: 1011
Output tokens for answer: 762
Total token:6917
Execution time for question: 45.04 seconds
Để trả lời câu hỏi về sự thay đổi tỷ lệ doanh thu từ các khoản vay thế chấp so với các khoản vay tiêu dùng tại ngân hàng ACB trong quý 2 năm 2024, chúng ta cần phân tích các kết quả từ các truy vấn SQL đã thực hiện.

### 1. Doanh thu từ các khoản vay thế chấp và tiêu dùng:

- **Doanh thu từ các khoản vay thế chấp:** 
  - Kết quả từ truy vấn cho thấy doanh thu từ các khoản vay thế chấp là **1,421,150.00**.
  
- **Doanh thu từ các khoản vay tiêu dùng:** 
  - Kết quả từ truy vấn cho thấy doanh thu từ các khoản vay tiêu dùng là **150,897.00**.

### 2. Tính toán tỷ lệ doanh thu:

Tỷ lệ doanh thu từ các khoản vay thế chấp so với các khoản vay tiêu dùng được tính bằng công thức sau:

\[
\text{Tỷ lệ doanh thu} = \left(\frac{\text{Doanh thu từ vay thế chấp}}{\text{Doanh thu từ vay tiêu dùng}}\right) \times 100
\]

Áp dụng vào số liệu:

\[
\

In [364]:
message = "Tìm những khoản nợ quá hạn của ACB quý 2 năm 2024"
text2sql_instance = GetAnswer(model="gpt-4o-mini", engine=sql_engine, db_structure=db_structure_data, acc_name=acc_name)

try:
    final_answer = text2sql_instance.get_result(message)
    if final_answer:
        print(final_answer)
except KeyError as e:
    print("Error:", e)

e:\llm_fin_2\venv\Lib\site-packages\langchain_community\utilities\sql_database.py:123: SAWarning: Did not recognize type 'vector' of column 'embedding'
  self._metadata.reflect(
e:\llm_fin_2\venv\Lib\site-packages\langchain_community\utilities\sql_database.py:123: SAWarning: Did not recognize type 'vector' of column 'bankembedding'
  self._metadata.reflect(


Input tokens for question: 5113
Input tokens for SQL queries: 187
Output tokens for answer: 716
Total token:6016
Execution time for question: 32.96 seconds
To address the user's question about identifying overdue debts of ACB (Asia Commercial Bank) for the second quarter of 2024, we can analyze the provided SQL queries and results.

### Breakdown of the SQL Queries

1. **First Query**: 
   ```sql
   SELECT "METADATA"."reportid"
   FROM "METADATA"
   JOIN "BANK" ON "METADATA"."bankid" = "BANK"."bankid"
   WHERE "BANK"."abbreviation" = 'ACB'
   AND "METADATA"."year" = 2024
   AND "METADATA"."quarter" = 2;
   ```
   - This query aims to fetch the `reportid` from the `METADATA` table for reports related to ACB for the second quarter of 2024. 
   - It joins the `METADATA` table with the `BANK` table to filter by the bank abbreviation 'ACB'.

2. **Second Query**: 
   ```sql
   SELECT "BALANCESHEET"."accountname", "BALANCESHEET"."amount"
   FROM "BALANCESHEET"
   JOIN "METADATA" ON "BALANCESH

In [366]:
!pip install PyYAML
import yaml

with open('db_structure.yaml', 'r', encoding='utf-8') as file:
    db_structure_data = yaml.safe_load(file)
    


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [259]:
print(db_structure_data.get("BANK", {}).get("columns", []))

[{'name': 'bankid', 'type': 'INTEGER', 'comment': 'Contains the list of bank IDs'}, {'name': 'bankname', 'type': 'TEXT', 'comment': 'Contains the list of Vietnamese bank names'}, {'name': 'symbol', 'type': 'TEXT', 'comment': 'Contains the stock codes of banks (e.g., BID, most of them have 3 capital letters)'}, {'name': 'abbreviation', 'type': 'TEXT', 'comment': 'Contains the English abbreviations of banks (e.g., VP Bank, BIDV)'}, {'name': 'banknameeng', 'type': 'VARCHAR(255)', 'comment': 'Contains the English names of banks'}, {'name': 'bankembedding', 'type': 'VECTOR', 'comment': 'Contains the embeddings of bank names'}]


In [214]:
def chain_of_thought_prompt(question):
    prompt = f"""
Bạn là một chuyên gia phân tích tài chính chuyên sâu. Hãy phân tích câu hỏi dưới đây và thực hiện các bước để trả lời chính xác và chi tiết.

Câu hỏi: "{question}"

Bước 1: Xác định loại vấn đề cần phân tích và chia nhỏ vấn đề
- Dịch câu hỏi sang tiếng anh
- Phân tích yêu cầu và chia nhỏ bài toán: [Xác định các vấn đề cụ thể cần được phân tích từ câu hỏi, chẳng hạn như các chỉ số tài chính hoặc thông tin tài sản liên quan]
- Từ đó, hãy tạo 1 danh sách với các câu lệnh nhỏ hơn với các vấn đề nhỏ đã được chia
Ví dụ: ['Hãy tìm lợi nhuận sau thuế và tổng vốn rồi tính tỷ lệ ROE rồi tìm ngân hàng có ROE lớn nhất năm 2024 quý 2","Hãy tìm tổng tài sản của ACB năm 2024 quý 2"]

Bước 2: Xác định thông tin cơ bản cần truy vấn từ vấn đề ở Bước 1
- Tên công ty: [Liệt kê rõ những công ty được đề cập, ví dụ: BIDV, Vietcombank, ACB]
- Năm tài chính: [Ghi rõ năm tài chính cần phân tích, ví dụ: 2024]
- Kỳ báo cáo: [Xác định kỳ báo cáo cụ thể, chẳng hạn quý 1, quý 2, quý 3, quý 4]
- Mục cụ thể trong báo cáo: [Dựa vào các vấn đề đã được chia nhỏ, xác định rõ mục cần tìm trong báo cáo tài chính. Ví dụ: doanh thu, lợi nhuận ròng, tổng tài sản, ROE (Return on Equity). Nếu liên quan đến các chỉ số tài chính, xác định các thành phần cụ thể để tính toán, ví dụ: lợi nhuận sau thuế, vốn chủ sở hữu]
- Loại báo cáo: [Xác định rõ loại báo cáo tài chính cần thiết, chẳng hạn Bảng cân đối kế toán, Báo cáo kết quả kinh doanh, Báo cáo lưu chuyển tiền tệ]

Bước 3: Trả về câu truy vấn SQL dựa trên các thông tin đã xác định ở bước 1, 2
- Với từng vấn đề trong danh sách trên, hãy tạo từng câu truy vấn sql cho từng vấn đề nhỏ
- Cấu trúc database của bạn: {json.dumps(db_structure)}
- Danh sách tên các tài khoản có sẵn: {json.dumps(acc_name)}
Đây là ví dụ: {load_example()}

Bước 4: Giải thích và kiểm tra lại lỗi truy vấn SQL
- Với từng vấn đề trong danh sách trên, hãy giải thích lại lệnh truy vấn.
- Sau khi giải thích, hãy đối chiếu lại với yêu cầu của câu hỏi và xác nhận xem đúng chưa
- Nếu đúng thì xác nhận và chuyển sang bước 5, nếu sai thì hãy sửa lại câu truy vấn đấy bằng cách thực hiện lại từ bước 1

Bước 5: Ghi lại câu truy vấn SQL cuối cùng 
- Sau khi đảm bảo các câu truy vấn đã đúng, hãy viết lại câu truy vấn theo fomart sau:
```final_sql
[Câu truy vấn sql cuối cùng]
```
Lưu ý: 
- Chỉ sử dụng dữ liệu từ database đã kết nối và đảm bảo truy vấn chính xác từng chỉ số tài chính được yêu cầu.
- Với lưu ý cột symbol thường chứa những tổ hợp 3 chữ cái viết hoa, còn abbreviation viết tắt cho những dạng ngắn hơn.
- Với những câu hỏi tìm số tài khoản chung hoặc tìm tài khoản cấp X mà không đề cập đến thời gian/ngân hàng, hãy tìm trong bảng ACCNO
- Bậc của số tài khoản được quyết định theo độ dài. 1 là cao nhất, càng dài càng thấp.(Ví dụ: tài khoản 10, 11, 12 là tài khoản con của 1)
- Những câu hỏi liệt kê, Không tự động giới hạn kết quả, hãy liệt kê hết.
Hãy làm tuần tự từ bước 1 và trả lời bằng tiếng việt
"""
    return prompt

In [215]:
def load_example():
        """Loads the example text from example.txt."""
        with open('example.txt', 'r', encoding='utf-8') as file:
            return file.read()

In [216]:
def get_sql_query(input_question):
    try:
        # Generate the prompt using the chain of thought method
        prompt_cot = chain_of_thought_prompt(input_question)
        # Create a chain to generate the SQL query
        sql_query_chain = create_sql_query_chain(llm, sql_database)
        # Invoke the chain and get the SQL query
        sql_query = sql_query_chain.invoke({"question": prompt_cot})
        # Extract the SQL query from the response
        return extract_sql_queries(sql_query)
    except Exception as e:
        print(f"Error generating SQL query: {e}")
        return None

def extract_sql_queries(sql_query):
    """
    Extracts all SQL queries from the response and returns them in a list.
    """
    pattern = r"```final_sql\s*(.*?)\s*```"
    matches = re.findall(pattern, sql_query, re.DOTALL)
    # Ensure unique queries by converting to a set and back to a list
    return list(set(match.strip() for match in matches)) if matches else []

In [217]:
llm = ChatOpenAI(model='gpt-4o-mini')
sql_database = SQLDatabase(engine=sql_engine)
question = 'Tìm số dư tài khoản tiền mặt, bạc, vàng và đá quý của NH TMCP Á Châu quý 2 năm 2024'
sql_queries = get_sql_query(question)

e:\llm_fin_2\venv\Lib\site-packages\langchain_community\utilities\sql_database.py:123: SAWarning: Did not recognize type 'vector' of column 'embedding'
  self._metadata.reflect(
e:\llm_fin_2\venv\Lib\site-packages\langchain_community\utilities\sql_database.py:123: SAWarning: Did not recognize type 'vector' of column 'bankembedding'
  self._metadata.reflect(


In [218]:
sql_queries

['SELECT "BALANCESHEET"."amount"\nFROM "METADATA"\nJOIN "BANK" ON "METADATA"."bankid" = "BANK"."bankid"\nJOIN "BALANCESHEET" ON "METADATA"."reportid" = "BALANCESHEET"."reportid"\nWHERE "BANK"."bankname" = \'Ngân hàng TMCP Á Châu\'\n  AND "BALANCESHEET"."accountname" = \'Cash, silver, gold and gemstones\'\n  AND "METADATA"."year" = 2024\n  AND "METADATA"."quarter" = 2\nLIMIT 1;']

In [219]:
def execute_sql_queries(sql_queries):
    """
    Execute each SQL query in the list and return the results in the order they were executed.
    """
    results = []  # List to store the results
    for query in sql_queries:
        try:
            # Execute the query using your database connection
            result = QuerySQLDataBaseTool(db=sql_database).run(query)
            results.append(result)  # Add the result to the list
        except Exception as e:
            print(f"Error executing query: {query}\nError: {e}")
            results.append(None)  # Add None if there is an error
    return results

In [221]:
execute_sql_queries(sql_queries)

['Error: (psycopg2.OperationalError) server closed the connection unexpectedly\n\tThis probably means the server terminated abnormally\n\tbefore or while processing the request.\nserver closed the connection unexpectedly\n\tThis probably means the server terminated abnormally\n\tbefore or while processing the request.\n\n[SQL: SELECT "BALANCESHEET"."amount"\nFROM "METADATA"\nJOIN "BANK" ON "METADATA"."bankid" = "BANK"."bankid"\nJOIN "BALANCESHEET" ON "METADATA"."reportid" = "BALANCESHEET"."reportid"\nWHERE "BANK"."bankname" = \'Ngân hàng TMCP Á Châu\'\n  AND "BALANCESHEET"."accountname" = \'Cash, silver, gold and gemstones\'\n  AND "METADATA"."year" = 2024\n  AND "METADATA"."quarter" = 2\nLIMIT 1;]\n(Background on this error at: https://sqlalche.me/e/20/e3q8)']

In [40]:
def get_result(question):
    system_role_template = """
    Given the following user question, corresponding SQL queries, and SQL results, provide a detailed answer.
    Question: {question}
    SQL Queries: {queries}
    SQL Results: {results}
    Answer:
    """

    try:
        # Generate the SQL query from the user question
        sql_queries = get_sql_query(question)
        if not sql_queries:
            raise ValueError("Failed to generate SQL query from the question.")

        # Execute all SQL queries and retrieve results
        sql_results = execute_sql_queries(sql_queries)

        # Format the prompt and generate the final answer
        answer_input = {
            "question": question,
            "queries": sql_queries,
            "results": sql_results
        }
        answer_prompt = PromptTemplate.from_template(system_role_template)
        answer_chain = answer_prompt | llm | StrOutputParser()

        # Generate and return the answer
        return answer_chain.invoke(answer_input)

    except Exception as e:
        print(f"Error: {e}")
        return None

In [41]:
get_result('Tìm số dư tài khoản tiền mặt, bạc, vàng và đá quý của NH TMCP Á Châu quý 2 năm 2024. Tìm ngân hàng ROE lớn nhất quý 3 năm 2024')

'To address the user’s inquiry regarding the balances and the Return on Equity (ROE) for the specified bank and quarter, we will break down the provided SQL queries and their results.\n\n### Part 1: Balances for NH TMCP Á Châu in Q2 2024\n\n#### SQL Query:\n```sql\nSELECT \n    "BANK"."bankname", \n    "BALANCESHEET"."amount"\nFROM \n    "METADATA"\nJOIN \n    "BANK" ON "METADATA"."bankid" = "BANK"."bankid"\nJOIN \n    "BALANCESHEET" ON "METADATA"."reportid" = "BALANCESHEET"."reportid"\nWHERE \n    "METADATA"."year" = 2024 \n    AND "METADATA"."quarter" = 2 \n    AND "BALANCESHEET"."accountname" = \'Cash, silver, gold and gemstones\'\n    AND "BANK"."bankname" = \'NH TMCP Á Châu\'\nLIMIT 1;\n```\n\n#### SQL Result:\n```\n[(\'NH TMCP Á Châu\', Decimal(\'6594138.00\'))]\n```\n\n#### Interpretation:\nThe result indicates that the balance for the account combining cash, silver, gold, and gemstones for the bank "NH TMCP Á Châu" in the second quarter of 2024 amounts to **6,594,138.00**. This

In [44]:
def get_result(question):
    system_role_template = """
    Given the following user question, corresponding SQL queries, and SQL results, provide a detailed answer.
    Question: {question}
    SQL Queries: {queries}
    SQL Results: {results}
    Answer:
    """
    sql_results = execute_sql_queries(sql_queries)
    # Format the prompt and generate the final answer
    answer_input = {
        "question": question,
        "queries": sql_queries,
        "results": sql_results
    }
    answer_prompt = PromptTemplate.from_template(system_role_template)
    answer_chain = answer_prompt | llm | StrOutputParser()

    # Generate and return the answer
    return answer_chain.invoke(answer_input)
    

In [31]:
get_result('Tìm số dư tài khoản tiền mặt, bạc, vàng và đá quý của NH TMCP Á Châu quý 2 năm 2024. Tìm ngân hàng ROE lớn nhất quý 3 năm 2024')

'To answer the user\'s question, we need to analyze two parts: the first part is about the cash, silver, gold, and gemstones balance of NH TMCP Á Châu for the second quarter of 2024, and the second part is to find the bank with the highest Return on Equity (ROE) for the third quarter of 2024.\n\n### Part 1: NH TMCP Á Châu\'s Cash, Silver, Gold, and Gemstones Balance for Q2 2024\n\nThe SQL query executed to find this information was:\n\n```sql\nSELECT \n    "BALANCESHEET"."accountname", \n    "BALANCESHEET"."amount"\nFROM \n    "METADATA"\nJOIN \n    "BALANCESHEET" ON "METADATA"."reportid" = "BALANCESHEET"."reportid"\nJOIN \n    "BANK" ON "METADATA"."bankid" = "BANK"."bankid"\nWHERE \n    "BANK"."bankname" = \'NH TMCP Á Châu\'\n    AND "METADATA"."year" = 2024 \n    AND "METADATA"."quarter" = 2 \n    AND "BALANCESHEET"."accountname" = \'Cash, silver, gold and gemstones\'\nLIMIT 5;\n```\n\nThe result of this query returned:\n\n```\n[(\'Cash, silver, gold and gemstones\', Decimal(\'659413

In [29]:
sql_results = execute_sql_queries(sql_queries)
sql_results

["[('NHTMCP Sài Gòn Thương Tín', Decimal('4.29241719562016751000'))]",
 "[('Cash, silver, gold and gemstones', Decimal('6594138.00')), ('Cash, silver, gold and gemstones', Decimal('6594138.00'))]"]

In [26]:
import pandas as pd
question = pd.read_excel('Test_question.xlsx')

In [336]:
class GetAnswer:
    def __init__(self, model="gpt-4o-mini", engine=None, db_structure=None, acc_name=None):
        self.model = model
        self.llm = ChatOpenAI(model=model)
        self.sql_engine = engine
        self.sql_database = SQLDatabase(engine=engine)
        self.db_structure = db_structure
        self.acc_name = acc_name

#Lower name để tìm đc, embedding name
    def chain_of_thought_prompt(self, question):
        prompt = f"""
    Bạn là một chuyên gia phân tích tài chính chuyên sâu. Hãy phân tích câu hỏi dưới đây và thực hiện các bước để trả lời chính xác và chi tiết.

    Câu hỏi: "{question}
    """
        return prompt
    
    def get_token_usage(self, text):
        """
        Returns the estimated token usage for the given text.
        """
        encoding = tiktoken.encoding_for_model(self.model)  # Ensure the model is specified

        # Calculate the number of tokens for the text
        return len(encoding.encode(text))
    
    def load_example(self):
        """Loads the example text from example.txt."""
        with open('example.txt', 'r', encoding='utf-8') as file:
            return file.read()
        
    def get_sql_query(self, input_question):
        try:
            # Generate the prompt using the chain of thought method
            prompt_cot = self.chain_of_thought_prompt(input_question)
            # Create a chain to generate the SQL query
            sql_query_chain = create_sql_query_chain(self.llm, self.sql_database)
            # Invoke the chain and get the SQL query
            sql_query = sql_query_chain.invoke({"question": prompt_cot})
            # Extract the SQL query from the response
            return self.extract_sql_queries(sql_query)
        except Exception as e:
            print(f"Error generating SQL query: {e}")
            return None

    def extract_sql_queries(self, sql_query):
        """
        Extracts all SQL queries from the response and returns them in a list.
        """
        pattern = r"```sql\s*(.*?)\s*```"
        matches = re.findall(pattern, sql_query, re.DOTALL)
        # Ensure unique queries by converting to a set and back to a list
        return list(set(match.strip() for match in matches)) if matches else []

    def execute_sql_queries(self, sql_queries):
        """
        Execute each SQL query in the list and return the results in the order they were executed.
        """
        results = []  # List to store the results
        for query in sql_queries:
            try:
                # Execute the query using your database connection
                result = QuerySQLDataBaseTool(db=self.sql_database).run(query)
                results.append(result)  # Add the result to the list
            except Exception as e:
                print(f"Error executing query: {query}\nError: {e}")
                results.append(None)  # Add None if there is an error
                
        return results
    

    def get_result(self, question):
        system_role_template = """
        Given the following user question, corresponding SQL queries, and SQL results, provide a detailed answer.
        Question: {question}
        SQL Queries: {queries}
        SQL Results: {results}
        Answer:
        """

        total_input_tokens = 0
        total_output_tokens = 0

        try:
            start_time = time.time()

            # Step 1: Calculate tokens for the question
            prompt_cot = self.chain_of_thought_prompt(question)
            input_tokens_question = self.get_token_usage(prompt_cot)
            total_input_tokens += input_tokens_question
            
            print(f"Input tokens for question: {input_tokens_question}")

            # Step 2: Generate SQL queries from the question
            sql_queries = self.get_sql_query(question)
            input_tokens_sql = sum(self.get_token_usage(query) for query in sql_queries)
            total_input_tokens += input_tokens_sql
            print(f"Input tokens for SQL queries: {input_tokens_sql}")

            if not sql_queries:
                raise ValueError("Failed to generate SQL query from the question.")

            # Step 3: Execute all SQL queries and get results
            sql_results = self.execute_sql_queries(sql_queries)

            # Step 4: Format prompt and create the final answer
            answer_input = {
                "question": question,
                "queries": sql_queries,
                "results": sql_results
            }
            answer_prompt = PromptTemplate.from_template(system_role_template)
            answer_chain = answer_prompt | self.llm | StrOutputParser()

            # Step 5: Generate and return the answer
            answer = answer_chain.invoke(answer_input)
            output_tokens_answer = self.get_token_usage(answer)
            total_output_tokens += output_tokens_answer
            print(f"Output tokens for answer: {output_tokens_answer}")

            # Total tokens
            print(f"Total token:{total_input_tokens+total_output_tokens}")
            end_time = time.time()  # End timing
            execution_time = end_time - start_time
            print(f"Execution time for question: {execution_time:.2f} seconds")


            return answer

        except Exception as e:
            print(f"Error: {e}")
            return None

In [335]:
def process_questions(file_path, sql_engine, db_structure, acc_name):
    """
    Process questions from an Excel file, convert them to SQL queries using GetAnswer, 
    execute the queries, and return the results.

    Args:
        file_path (str): Path to the Excel file containing questions.
        sql_engine: SQLAlchemy engine for database connection.
        db_structure: Database structure to assist in SQL generation.
        acc_name (str): Account name or related database information.

    Returns:
        pd.DataFrame: DataFrame containing questions, SQL queries, and results.
    """
    # Read the file containing questions
    questions = pd.read_excel(file_path)
    
    # List to store results
    results = []
    
    # Iterate through each question in the 'question' column
    for question in questions['Question']:
        start_time = time.time()
        # Create an instance of GetAnswer
        text2sql_instance = GetAnswer(
            model="gpt-4o-mini", 
            engine=sql_engine, 
            db_structure=db_structure, 
            acc_name=acc_name
        )
        
        # Generate SQL query from the question
        prompt_cot = text2sql_instance.chain_of_thought_prompt(question)
        sql_query = text2sql_instance.get_sql_query(question)
        
        # Execute the SQL query and get the result (if a query was generated)
        sql_results = (
            text2sql_instance.execute_sql_queries(sql_query) 
            if sql_query else None
        )
        end_time = time.time()
        running_time = end_time - start_time
        # Calculate token usage
        total_input_tokens = text2sql_instance.get_token_usage(prompt_cot) + \
                             sum(text2sql_instance.get_token_usage(q) for q in sql_query) if sql_query else 0
        # Convert sql_results to string if it's not None
        total_output_tokens = text2sql_instance.get_token_usage(str(sql_results)) if sql_results else 0
        total_tokens = total_input_tokens + total_output_tokens
        
        # Append the question, SQL query, and results to the list
        results.append({
            'question': question,
            'sql_query': sql_query,
            'result': sql_results,
            'total input token': total_input_tokens,
            'total out token': total_output_tokens,
            'total token': total_tokens,
            'running time': running_time
        })
        
    # Create a DataFrame from the results
    result_df = pd.DataFrame(results)
    result_df.to_excel('results_non_cot.xlsx', index=False)

In [337]:
result_df = process_questions('Test_question.xlsx', sql_engine, db_structure_data, acc_name)

e:\llm_fin_2\venv\Lib\site-packages\langchain_community\utilities\sql_database.py:123: SAWarning: Did not recognize type 'vector' of column 'embedding'
  self._metadata.reflect(
e:\llm_fin_2\venv\Lib\site-packages\langchain_community\utilities\sql_database.py:123: SAWarning: Did not recognize type 'vector' of column 'bankembedding'
  self._metadata.reflect(
e:\llm_fin_2\venv\Lib\site-packages\langchain_community\utilities\sql_database.py:123: SAWarning: Did not recognize type 'vector' of column 'embedding'
  self._metadata.reflect(
e:\llm_fin_2\venv\Lib\site-packages\langchain_community\utilities\sql_database.py:123: SAWarning: Did not recognize type 'vector' of column 'bankembedding'
  self._metadata.reflect(
e:\llm_fin_2\venv\Lib\site-packages\langchain_community\utilities\sql_database.py:123: SAWarning: Did not recognize type 'vector' of column 'embedding'
  self._metadata.reflect(
e:\llm_fin_2\venv\Lib\site-packages\langchain_community\utilities\sql_database.py:123: SAWarning: Did 